In [46]:
import numpy as np
from tabulate import tabulate
from scipy.sparse.linalg import eigsh
from scipy.sparse import coo_array, kron

In [47]:
i = coo_array(np.array([[1,0],[0,1]]))
z = coo_array(np.array([[1,0],[0,-1]]))
x = coo_array(np.array([[0,1],[1,0]]))
print(z.size)
print(z.shape)

2
(2, 2)


In [48]:
kron(z, i).toarray()

array([[ 1,  0,  0,  0],
       [ 0,  1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0, -1]])

In [43]:
def tensor(a, b):
    block_result = np.tensordot(a, b, 0)
    for row in range(block_result.shape[0]):
        crow = np.concatenate(block_result[row], axis=1)
        if row == 0:
            result = crow
        else:
            result = np.concatenate((result, crow), axis=0)
    return result
print(tabulate(tensor(z, i), tablefmt="fancy_grid"))
print(tabulate(np.kron(z, i), tablefmt="fancy_grid"))

IndexError: tuple index out of range

In [89]:
def tensorIright(matrix):
    i = np.array([[1,0],[0,1]])
    new_matrix = tensor(matrix, i, 0)
    print(tabulate(new_matrix, tablefmt="fancy_grid"))
    print(new_matrix.shape[0])
tensorIright(z)

TypeError: tensor() takes 2 positional arguments but 3 were given

In [21]:
def Hmatrix(num_qubits):
    h_zz = -1
    h_z = 1
    h_x = 1
    i = np.array([[1,0],[0,1]])
    z = np.array([[1,0],[0,-1]])
    x = np.array([[0,1],[1,0]])
    H = np.zeros((2**num_qubits, 2**num_qubits))
    #zz
    zz_term = tensor(z, z)
    num_left = 0
    num_right = num_qubits - 2
    for sum_index in range(num_qubits - 1):
        for j in range(num_right):
            zz_term = tensor(zz_term, i)
        for j in range(num_left):
            zz_term = tensor(i, zz_term)
        H += h_zz * zz_term
        num_left += 1
        num_right -= 1
        zz_term = tensor(z, z)
    #z
    z_term = z.copy()
    num_left = 0
    num_right = num_qubits - 1
    for sum_index in range(num_qubits):
        for j in range(num_right):
            z_term = tensor(z_term, i)
        for j in range(num_left):
            z_term = tensor(i, z_term)
        H += h_z * z_term
        num_left += 1
        num_right -= 1
        z_term = z.copy()
    #x
    x_term = x.copy()
    num_left = 0
    num_right = num_qubits - 1
    for sum_index in range(num_qubits):
        for j in range(num_right):
            x_term = tensor(x_term, i)
        for j in range(num_left):
            x_term = tensor(i, x_term)
        H += h_x * x_term
        num_left += 1
        num_right -= 1
        x_term = x.copy()
    return H

[[ 1  0]
 [ 0 -1]]


array([[ 1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  3.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  1., -1.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  1.,  1.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  1.,  0., -1.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  1., -5.]])

In [34]:
matrix = Hmatrix(3)
min_eigenvalue, min_eigenvectors = eigsh(matrix, k=1, which='SA')
print(min_eigenvalue)

[[ 1  0]
 [ 0 -1]]
[-5.65387565]
